In [52]:
import pandas as pd
from sklearn.utils import shuffle
import os 

In [53]:
from data_preprocessing import *

# Preprocess


## read & filter

In [54]:
data_dir = '/home/yhyeung2/CoV_Encoder_HA/src/data/'

headers = ['Name', 'VH_AA','VL_AA', 'CDRH1_AA','CDRH2_AA','CDRH3_AA','CDRL1_AA','CDRL2_AA','CDRL3_AA']

All = pd.read_excel(data_dir+'raw/all_paired_antibodies_from_GB_v6.xlsx', usecols=headers)


In [55]:

# experiments 1 and 2

HA = pd.read_excel(data_dir+'raw/HA_Abs_v14.10.1.xlsx', usecols=headers)
X_df, y_df = decouple_haothers(All, HA)

# experiment 3

# HA_stemhead = pd.read_table(data_dir+'raw/Ab_for_model_20230119.tsv')
# HAstem = HA_stemhead.loc[HA_stemhead['Binds_to']=='HA:Stem']; HAstem=HAstem.drop('Binds_to',axis=1)
# HAhead = HA_stemhead.loc[HA_stemhead['Binds_to']=='HA:Head']; HAhead=HAhead.drop('Binds_to',axis=1)
# X_df, y_df = process(All, HAhead, HAstem)

# experiment 4

# HAstem = pd.read_excel(data_dir+'raw/HA_Abs_v14.10.1_stem.xlsx', usecols=headers)
# HAhead = pd.read_excel(data_dir+'raw/HA_Abs_v14.10.1_head.xlsx', usecols=headers)
# X_df, y_df = process(All, HAhead, HAstem)


In [59]:

# display('shape', X_df.shape, y_df.shape)
X_df.to_csv(f'{data_dir}/output/HA_X.csv', index=False)
y_df.to_csv(f'{data_dir}/output/HA_y.csv', index=False)

display(X_df, y_df['binding'].value_counts())


,Name,VH_AA,VL_AA,CDRH1_AA,CDRH2_AA,CDRH3_AA,CDRL1_AA,CDRL2_AA,CDRL3_AA
0,9F2,EVQLLESGGGLAQPGGSLRLSCAASGFTFSSYAMSWVRQAPGKGLE...,QSVLTQPPSVSAAPGQKVTISCSGSSSNIGRNYVSWYQHLPGTAPK...,GFTFSSYA,INYSGGST,AKGLMKPPHIGSLFGVVRIGPFDH,SSNIGRNY,DNN,GTWDSSLSGVV
1,K1B12,EIQLAQSEAEVKKPGAAVKVSCKTSGYIFTTFGISWVRQAPGQGLE...,QSVLTQPPSVSGTPGQRVTLSCSGTSSNIGAGFDVHWYQQVPGAAP...,GYIFTTFG,VSTRNGDT,VRDGRYGGYSQISGPNYSLYHGMDV,SSNIGAGFD,SDN,QSYDSSLSGSRV
2,10I,LESGGDLVQPGGSLRLSCEASGFTFGSYAMTWVRQAPGKGLEWVSS...,ELTQSPSSLSASVGDRVTITCRASQSISSYLNWYQQKPGKAPKLLI...,GFTFGSYA,FSANGDFA,AKAGRILGVVLWYSLYYGFDV,QSISSY,AAS,QQSYSTPFT
3,7F,LESGPGLVKPSETLSLTCTVSGGAMGDGDYYWSWIRQHPGKGLEWL...,ELTQSPGTLSLSPGERATLSCRASQSVSSNFLAWYQQKPGQAPRLL...,GGAMGDGDYY,IYYSGTT,ARGRAALFGSESYPLDH,QSVSSNF,GAS,QHYGTPRT
4,JMpFv2-40,VKLLEESGGGLVQPGGSLRLSCAASASTFSRYAVDWVRQAPGKGLE...,MAELTQSPTTLSVSPGERATLSCRASQSDANYLAWYQQKAGQAPRL...,ASTFSRYA,ESAGGT,ATRGEYYYYGMDV,QSDANY,DAS,QYYGDSPPIT
...,...,...,...,...,...,...,...,...,...
10382,310-18G11,VQLVQSGAEVKKPGSSVKVSCKASGGTFSNYAINWVRQAPGQGLEW...,TLSASVGDRVTITCRASQGISNHLAWYQQKPGTVPKLLIYAASTLQ...,GGTFSNYA,IIPIFGTP,ATARDSGWYYHAIDV,QGISNH,AAS,QKYNSAPPGIT
10383,310-18G7,VQLAQSGAELKKPGSSVKVSCKASGGPFSSNGVSWVRQAPGQGLEW...,QSVLTQPPSASGTPGQRVTISCSGSSSNIGSNTVNWYQQLPRTAPK...,GGPFSSNG,IIAIFGTT,ARSSNYYYGPGSWFDP,SSNIGSNT,SNN,AVWDGSLNGWV
10384,310-18H10,QVQLVQSGAEVKKPGSSVKVSCKASGGTFSSNAVTWVRQAPGQGLE...,TLSLSPGDRATLSCRASQSVGSNFLAWYQQKPGQAPRLLIYGASSR...,GGTFSSNA,IIGMFGVT,ATKGHYYESSLDY,QSVGSNF,GAS,QQYGSSPRT
10385,310-18H3,VQLVQSGAEVKKPGSSVKVSCKTSGRTFSSYAISWVRQAPGQGLEW...,VLTQPRSVSGSPGQSVTISCTGTSSDVGAYKYVSWYQQLPGKAPRL...,GRTFSSYA,IIGMFGTT,ARGTTYYHGPAEF,SSDVGAYKY,DVN,CSFAGNTYV


False    8968
True     1419
Name: binding, dtype: int64

## ANARCI

Using [ANARCI](https://github.com/oxpig/ANARCI) (Antibody Numbering and Antigen Receptor ClassIfication) for sequence alignment

In [51]:
'''
bash script ('awk') to convert .csv into .fasta file as ANARCI input 
by selecting desired columns with index(es)

e.g.
given .csv with columns {Name,VH_AA,VL_AA,CDRH1_AA,CDRH2_AA,CDRH3_AA,CDRL1_AA,CDRL2_AA,CDRL3_AA}
$1 => col[0] == 'Name'
$3 => col[4] == 'VL_AA'
'''

!awk -F , '{print ">"$1"\n"$2$3}' data/output/HA_X.csv > data/output/HA_X.fasta
!sed -i '1,2d' data/HA_X.fasta 
!ANARCI --scheme chothia --csv -i data/output/HA_X.fasta -o data/output/HA-ANARCI


In [80]:
# assemble
    
x_h = pd.read_csv(f'{data_dir}/output/HA-ANARCI_H.csv')
x_l = pd.read_csv(f'{data_dir}/output/HA-ANARCI_KL.csv')

y_original = pd.read_csv(f'{data_dir}/output/HA_y.csv')

x = anarci_process(x_h, x_l, y_original)
y = y_original.set_index('Name')
y = y.loc[x['Id']]

x.to_csv(f'{data_dir}/output/HA-ANARCI_X.csv', index=False)
y.to_csv(f'{data_dir}/output/HA-ANARCI_y.csv')


,Id,VH_AA,VL_AA
0,24.0|VRC_310_cohort_donor_16,QVQLVQ-SQTEVKKP--G--ASVK-VSCKASGYTFTR---------...,DIQMTQSPSSLSASVGDRVTITC-RASQDIS---------RWLAW-...
1,V3-1G10,QVQLVQ-SGAEVKKP--G--STVK-VSCEASGVTFNH---------...,EIIMTQSPATLSLSPGERVTLSC-RASQSVG---------TNLAW-...
2,CR6329,EVQLVQ-SGAEVKKP--G--SSVK-VSCKASGGIFRS---------...,EIVLTQSPGTLSLSPGERATLSC-RASQSVSS--------NYLGW-...
3,SFV009_3D04,EVQLLE-SGGGLVQP--G--ASLR-LSCAASGFSFKD---------...,EIVMTQSPATVSVSPGERATLSC-RASQSVN---------SDLVW-...
4,A66,-------------KP--G--SSVK-VSCKASGGPFSM---------...,EIVLTQSPATLSLSPGERATLSC-RASQSVS---------SYLAW-...
...,...,...,...
9270,Z1800M-30Jan08-5HB3-1,QVQLVQ-SGAEVKKP--G--SSVK-VSCKASGGTFSS---------...,--VLTQPPS-VSVAPGKTARITC-GGDNIGS---------KSVHW-...
9271,Z1800M-30Jan08-5HB6-2,EVQLVE-SGGGLVQP--G--GSLR-LSCAASGFTFIS---------...,EIVLTQSPATLSMSPGERATLSC-RASQSVA---------SNLAW-...
9272,Z1800M-30Jan08-6HB11-3,EVQLVQ-SGSEVKRP--G--SSVK-VSCKASGGTFSI---------...,DIQLTQSPSSLSASVGDRVTITC-RASQGIR---------NDLGW-...
9273,Z1800M-30Jan08-6HF9-5,EVQLVE-SGAEVKKP--G--SSVK-VSCKASGGTFNI---------...,DIVMTQSPSSLSASVGDRVTITC-RASQGIR---------NDLGW-...


,binding
Name,
24.0|VRC_310_cohort_donor_16,stem
V3-1G10,stem
CR6329,stem
SFV009_3D04,stem
A66,stem
...,...
Z1800M-30Jan08-5HB3-1,others
Z1800M-30Jan08-5HB6-2,others
Z1800M-30Jan08-6HB11-3,others


binding
others     9077
head        103
stem         95
dtype: int64

## calibrate exp 3 and 4

ensure both have same set of negative cases ('others' class) for fair comparison

In [ ]:

def calibrate(a, b):
    
    def intersect(s1, s2):
        return pd.Series(list(set(s1).intersection(set(s2))))

    def uni(s1, s2):
        return pd.Series(list(set(s1).union(set(s2))))
    
    (aX, ay), (bX, by) = a, b
    
    a = aX.copy()
    a['binding'] = ay['binding']
    
    b = bX.copy()
    b['binding'] = by['binding']
    
    assert ~(a['Id'].duplicated().all())
    assert ~(b['Id'].duplicated().all())
    
    a_positive_id = a.loc[a['binding']!='others', 'Id']
    a_negative_id = a.loc[a['binding']=='others', 'Id']
    
    b_positive_id = b.loc[b['binding']!='others', 'Id']
    b_negative_id = b.loc[b['binding']=='others', 'Id']
    
    positive_id = intersect(a_positive_id, b_positive_id)
    
    negative_id = intersect(a_negative_id, b_negative_id)
    negative_id = negative_id.loc[~negative_id.isin(positive_id)]
    
    a_id = a['Id'].isin(pd.concat([a_positive_id, negative_id]))
    b_id = b['Id'].isin(pd.concat([b_positive_id, negative_id]))
    display(a_id.value_counts(), b_id.value_counts())
    
    a = a.loc[a_id,:].reset_index(drop=True)
    ay = a.loc[:, ['Id', 'binding']]
    
    b = b.loc[b_id,:].reset_index(drop=True)
    by = b.loc[:, ['Id', 'binding']]
    
    return (a, ay), (b, by)
    

In [ ]:

exp3x = pd.read_csv(f'{data_dir}experiment3/HA-ANARCI_X.csv')
exp3y = pd.read_csv(f'{data_dir}experiment3/HA-ANARCI_y.csv')
exp4x = pd.read_csv(f'{data_dir}experiment4/HA-ANARCI_X.csv')
exp4y = pd.read_csv(f'{data_dir}experiment4/HA-ANARCI_y.csv')

(exp3x, exp3y), (exp4x, exp4y) = calibrate((exp3x, exp3y), (exp4x, exp4y))

exp3x.to_csv(f'{data_dir}experiment3/HA-ANARCI_X.csv')
exp3y.to_csv(f'{data_dir}experiment3/HA-ANARCI_y.csv')
exp4x.to_csv(f'{data_dir}experiment4/HA-ANARCI_X.csv')
exp4y.to_csv(f'{data_dir}experiment4/HA-ANARCI_y.csv')
